##Analisi Preliminare

In [ ]:
!pip install scikit-plot

In [ ]:
!pip install kneed

In [ ]:
!pip install lime

In [ ]:
import sklearn
import lime
import lime.lime_tabular
from lime import lime_tabular

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
import scipy
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Librerie
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import scikitplot as skplt

from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from numpy import random

from sklearn.neighbors import KNeighborsClassifier

from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV

from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv("vehicle.csv")

In [ ]:
data.head(20)

In [ ]:
data.describe()

In [ ]:
data['Class'].value_counts()

In [ ]:
y = data["Class"]
X = data[['COMPACTNESS','CIRCULARITY','DISTANCE_CIRCULARITY','RADIUS_RATIO',"PR.AXIS_ASPECT_RATIO","MAX.LENGTH_ASPECT_RATIO","SCATTER_RATIO",
          "ELONGATEDNESS","PR.AXIS_RECTANGULARITY","MAX.LENGTH_RECTANGULARITY","SCALED_VARIANCE_MAJOR","SCALED_VARIANCE_MINOR",
          "SCALED_RADIUS_OF_GYRATION","SKEWNESS_ABOUT_MAJOR","SKEWNESS_ABOUT_MINOR","KURTOSIS_ABOUT_MAJOR","KURTOSIS_ABOUT_MINOR","HOLLOWS_RATIO"]]

In [ ]:
# initialising the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
# creating the bar plot
ax=plt.subplots(1,1,figsize=(10,8))
sns.countplot('Class',data=data)
plt.title("Class Count")
plt.show()

In [ ]:
# Matrice di correlazione
corr_data = data.corr(method='pearson')

plt.figure(figsize=(10, 10))
sns.heatmap(corr_data, annot=True)
plt.show()

In [ ]:
classes=["van","saab","bus","opel"]
features=['COMPACTNESS','CIRCULARITY','DISTANCE_CIRCULARITY','RADIUS_RATIO',"PR.AXIS_ASPECT_RATIO","MAX.LENGTH_ASPECT_RATIO","SCATTER_RATIO",
          "ELONGATEDNESS","PR.AXIS_RECTANGULARITY","MAX.LENGTH_RECTANGULARITY","SCALED_VARIANCE_MAJOR","SCALED_VARIANCE_MINOR","SCALED_RADIUS_OF_GYRATION",
          "SKEWNESS_ABOUT_MAJOR","SKEWNESS_ABOUT_MINOR","KURTOSIS_ABOUT_MAJOR","KURTOSIS_ABOUT_MINOR","HOLLOWS_RATIO"]

In [ ]:
feature_names = list(X.columns)

##Regressione Logistica

In [ ]:
# Modello
logistic = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# Allenamento sui dati 
logistic.fit(X_train,y_train)
# Previsione sui dati di test
p_test=logistic.predict(X_test)

In [ ]:
# Precisione
acc = accuracy_score(y_test, p_test) 
print(acc)

0.7283464566929134


In [ ]:
report=classification_report(p_test, y_test) 
print(report)

In [ ]:
# Matrice di Confusione
skplt.metrics.plot_confusion_matrix(p_test,y_test)
plt.show()

In [ ]:
resultsDf=pd.DataFrame({'Model':['Logistic'],'Accuracy': logistic.score(X_test , y_test)},index={'1'})
resultsDf=resultsDf[['Model','Accuracy']]
resultsDf

In [ ]:
# importing the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import tree

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(X_train, mode="classification", class_names=classes, feature_names= features)

idx = 5
explanation = explainer.explain_instance(X_test[idx], logistic.predict_proba, num_features=len(features),top_labels=4)

explanation.show_in_notebook()

In [ ]:
explanation.show_in_notebook()

## KNN

In [ ]:
parameters = {
     "n_neighbors": range(1, 100),
     "weights": ["uniform", "distance"],
 }
knn = GridSearchCV(KNeighborsClassifier(), parameters)
knn.fit(X_train, y_train)
print(knn.best_params_)
p_test = knn.predict(X_test)

{'n_neighbors': 3, 'weights': 'distance'}


In [ ]:
# Precisione
acc = accuracy_score(y_test, p_test) 
print(acc)

In [ ]:
report=classification_report(p_test, y_test) 
print(report)

In [ ]:
# Matrice di Confusione
skplt.metrics.plot_confusion_matrix(p_test,y_test)
plt.show()

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(X_train, mode="classification", class_names=classes, feature_names= features)

idx = 5
explanation = explainer.explain_instance(X_test[idx], knn.predict_proba, num_features=len(features),top_labels=4)

explanation.show_in_notebook()

In [ ]:
tempResultsDf = pd.DataFrame({'Model':['KNN'], 'Accuracy': knn.score(X_test, y_test)},index={'2'})
resultsDf = pd.concat([resultsDf, tempResultsDf])
resultsDf = resultsDf[['Model','Accuracy']]
resultsDf

## Multiclass Support Vector Machines

In [ ]:
params_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid',"linear"]}
svm = GridSearchCV(SVC(probability=True), params_grid, cv=5)
svm.fit(X_train,y_train)
p_test=svm.predict(X_test)

In [ ]:
# View the accuracy score
print('Best score for training data:', svm.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm.best_estimator_.C,"\n") 
print('Best Kernel:',svm.best_estimator_.kernel,"\n")
print('Best Gamma:',svm.best_estimator_.gamma,"\n")

In [ ]:
p_test

In [ ]:
report=classification_report(p_test, y_test) 
print(report)

In [ ]:
# Matrice di Confusione
skplt.metrics.plot_confusion_matrix(p_test,y_test)
plt.show()

In [ ]:
tempResultsDf = pd.DataFrame({'Model':['SVM'], 'Accuracy': svm.score(X_test, y_test)},index={'2'})
resultsDf = pd.concat([resultsDf, tempResultsDf])
resultsDf = resultsDf[['Model','Accuracy']]
resultsDf

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(X_train, mode="classification", class_names=classes, feature_names= features)

idx = 5
explanation = explainer.explain_instance(X_test[idx], svm.predict_proba, num_features=len(features),top_labels=4)

explanation.show_in_notebook()

## Risultati finali

In [ ]:
resultsDf=pd.DataFrame({'Model':['Logistic'],'Accuracy': logistic.score(X_test , y_test)},index={'1'})
resultsDf=resultsDf[['Model','Accuracy']]
resultsDf

In [ ]:
#Store the accuracy results for each kernel in a dataframe for final comparison
tempResultsDf = pd.DataFrame({'Model':['Decision Tree'], 'Accuracy': DTC.score(X_test, y_test)},index={'2'})
resultsDf = pd.concat([resultsDf, tempResultsDf])
resultsDf = resultsDf[['Model','Accuracy']]
resultsDf

In [ ]:
#Store the accuracy results for each kernel in a dataframe for final comparison
tempResultsDf = pd.DataFrame({'Model':['KNN'], 'Accuracy': knn.score(X_test,y_test)},index={'3'})
resultsDf = pd.concat([resultsDf, tempResultsDf])
resultsDf = resultsDf[['Model','Accuracy']]
resultsDf

In [ ]:
#Store the accuracy results for each kernel in a dataframe for final comparison
tempResultsDf = pd.DataFrame({'Model':['SVM'], 'Accuracy': svm.score(X_test, y_test)},index={'4'})
resultsDf = pd.concat([resultsDf, tempResultsDf])
resultsDf = resultsDf[['Model','Accuracy']]
resultsDf

## Bagging

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [ ]:
models = []
scores = []

for n_estimators in range(1,101):

    # Create bagging classifier
    clf = BaggingClassifier(n_estimators = n_estimators, random_state = 100)

    # Fit the model
    clf.fit(X_train, y_train)

    # Append the model and score to their respective list
    models.append(clf)
    scores.append(accuracy_score(y_true = y_test, y_pred = clf.predict(X_test)))

In [ ]:
import matplotlib.pyplot as plt

# Generate the plot of scores against number of estimators
plt.figure(figsize=(9,6))
plt.plot(range(1,101), scores)

# Adjust labels and font (to make visable)
plt.xlabel("n_estimators", fontsize = 18)
plt.ylabel("score", fontsize = 18)
plt.tick_params(labelsize = 16)

# Visualize plot
plt.show()

In [ ]:
bag=BaggingClassifier(DecisionTreeClassifier(criterion = "gini", random_state = 100),
                     n_estimators=18)
bag.fit(X_train,y_train)
p_test=bag.predict(X_test)

In [ ]:
print(classification_report(y_test,p_test))

In [ ]:
# Matrice di Confusione
skplt.metrics.plot_confusion_matrix(p_test,y_test)
plt.show()

In [ ]:
import seaborn as sns
features =['COMPACTNESS','CIRCULARITY','DISTANCE_CIRCULARITY','RADIUS_RATIO',"PR.AXIS_ASPECT_RATIO","MAX.LENGTH_ASPECT_RATIO","SCATTER_RATIO","ELONGATEDNESS","PR.AXIS_RECTANGULARITY","MAX.LENGTH_RECTANGULARITY","SCALED_VARIANCE_MAJOR","SCALED_VARIANCE_MINOR","SCALED_RADIUS_OF_GYRATION","SKEWNESS_ABOUT_MAJOR","SKEWNESS_ABOUT_MINOR","KURTOSIS_ABOUT_MAJOR","KURTOSIS_ABOUT_MINOR","HOLLOWS_RATIO"]
all_feat_imp_df = pd.DataFrame(data=[tree.feature_importances_ for tree in 
                                     h],
                               columns=features)
(sns.boxplot(data=all_feat_imp_df)
        .set(title='Feature Importance Distributions',
             ylabel='Importance'));

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(X_train, mode="classification", class_names=classes, feature_names= features)

idx = 5
explanation = explainer.explain_instance(X_test[idx], bag.predict_proba, num_features=len(features),top_labels=4)

explanation.show_in_notebook()

In [ ]:
with plt.style.context("ggplot"):
    explanation.as_pyplot_figure()

In [ ]:
tempResultsDf = pd.DataFrame({'Model':['Bagging'], 'Accuracy': bag.score(X_test, y_test)},index={'2'})
resultsDf = pd.concat([resultsDf, tempResultsDf])
resultsDf = resultsDf[['Model','Accuracy']]
resultsDf